In [1]:
import numpy as np
from PIL import Image
from scipy.ndimage import label
import matplotlib.pyplot as plt
import os
import pandas as pd

In [2]:
def mask(outarray,inarray):
    
    outarray = outarray
    inarray = inarray
    
    condition = (outarray==255)
    new_value = 1
    outarray[condition] = new_value

    mult_red = np.multiply(outarray,inarray[:,:,0])
    mult_green = np.multiply(outarray,inarray[:,:,1])
    mult_blue = np.multiply(outarray,inarray[:,:,2])

    return outarray, mult_red, mult_green, mult_blue

In [3]:
def isolate_comps(num_comp,outarray,mult_red, mult_green, mult_blue,labeled_out):
    comp_red = [[] for _ in range(num_comp)] #list of red pixels per component [[comp1 red pixels], [comp 2 red pixels]....]
    comp_green = [[] for _ in range(num_comp)] #list of green pixels per component [[comp1 green pixels], [comp 2 green pixels]....]
    comp_blue = [[] for _ in range(num_comp)] #list of blue pixels per component [[comp1 blue pixels], [comp 2 blue pixels]....]

    comps = [[] for _ in range(num_comp)] #[[[comp1_x1,comp1_y1],[comp1_x2,comp1_y2]...],[[comp2_x1,comp2_y1],[comp2_x2,comp2_y2]...]...]

    for x in range(outarray.shape[0]):
        for y in range(outarray.shape[1]):

            comp_id = labeled_out[x,y]

            redPix = mult_red[x,y]
            greenPix = mult_green[x,y]
            bluePix = mult_blue[x,y]

            if comp_id!=0:
                ind = [x,y]
                comps[comp_id - 1].append(ind)
                comp_red[comp_id - 1].append(redPix)
                comp_green[comp_id - 1].append(greenPix)
                comp_blue[comp_id - 1].append(bluePix)
    return comps,comp_red,comp_green,comp_blue

In [4]:
def count(num_comp,comp_red,comp_green,comp_blue,red_thresh,green_thresh,blue_thresh):
    red_count = 0
    green_count = 0
    blue_count = 0
    red_green_count = 0
    for i in range(num_comp):
        red_max = max(comp_red[i])
        green_max = max(comp_green[i])
        blue_max = max(comp_blue[i])
        if (blue_max>blue_thresh):
            blue_count+=1
        if (red_max > red_thresh) & (green_max > green_thresh):
            red_green_count+=1
        elif (red_max > red_thresh):
            red_count+=1
        elif (green_max > green_thresh):
            green_count+=1
    return red_count,green_count,blue_count,red_green_count

In [22]:
#find threshold based on one image

red_thresh_list = list(range(0, 100, 1))
green_thresh_list = list(range(0, 100, 1))
blue_thresh = 20

true_red = 12
true_green = 40
true_red_green = 544


im = Image.open('') #(2048,2448) mask of 0s and 255s, output from MIA
outarray = np.array(im)

im = Image.open('') #(2048,2448,3) rgb input image
inarray = np.array(im)

outarray, mult_red, mult_green, mult_blue = mask(outarray,inarray)

structure = np.ones((3, 3), dtype=int)
labeled_out, num_comp = label(outarray,structure)

comps,comp_red,comp_green,comp_blue = isolate_comps(num_comp,outarray,mult_red, mult_green, mult_blue,labeled_out)

errors = []
threshs = []

for redT in red_thresh_list:
    for greenT in green_thresh_list:
        red_thresh = redT
        green_thresh = greenT
        red_count,green_count,blue_count,red_green_count = count(num_comp,comp_red,comp_green,comp_blue,red_thresh,green_thresh,blue_thresh)
        error = abs(true_red - red_count) + abs(true_green - green_count) + abs(true_red_green - red_green_count)
        errors.append(error)
        threshs.append([red_thresh,green_thresh])

min_value = min(errors)
#print(min_value)
min_index = errors.index(min_value)
thresh_min = threshs[min_index]
#print(thresh_min)

# Find the minimum value in the errors list
min_value = min(errors)

# Get the indices of the minimum value in the errors list
min_indices = [index for index, value in enumerate(errors) if value == min_value]

# Initialize max sum and corresponding thresh
max_sum = float('-inf')
best_thresh = None

# Iterate through the corresponding threshs for the minimum error indices
for i in min_indices:
    red_thresh, green_thresh = threshs[i]
    current_sum = red_thresh + green_thresh
    
    # Update if current sum is greater than max_sum
    if current_sum > max_sum:
        max_sum = current_sum
        best_thresh = (red_thresh, green_thresh)

# Output the results
print("Minimum error value:", min_value)
print("Maximum sum of thresholds:", max_sum)
print("Corresponding red_thresh and green_thresh:", best_thresh)
#red = 67, green = 39. use +-5 to narrow search

Minimum error value: 8
Maximum sum of thresholds: 106
Corresponding red_thresh and green_thresh: (67, 39)


In [7]:
#correct image number--Not all NCIH2170 control images have ground truth FISH probe data, select subset that do

import pandas as pd
import shutil
import os

# Load the control DataFrame
#control_df = pd.read_csv('')

# Define folder paths
#folder_path_in = ''
#folder_path_out_corrected = ''
#folder_path_out_labels = ''
#folder_path_out_labels_corrected = ''

# Create corrected folders if they don't exist
os.makedirs(folder_path_out_corrected, exist_ok=True)
os.makedirs(folder_path_out_labels_corrected, exist_ok=True)

# Get the list of image names from the control DataFrame
image_names = control_df['Image name'].tolist()

# Copy images from 'images used' to 'images used corrected'
for image_name in image_names:
    src_image_path = os.path.join(folder_path_in, image_name)
    dst_image_path = os.path.join(folder_path_out_corrected, image_name)
    if os.path.isfile(src_image_path):  # Check if the source file exists
        shutil.copy(src_image_path, dst_image_path)

# Copy tif labels from 'tif labels filled' to 'tif labels filled corrected'
for image_name in image_names:
    src_label_path = os.path.join(folder_path_out_labels, image_name)
    dst_label_path = os.path.join(folder_path_out_labels_corrected, image_name)
    if os.path.isfile(src_label_path):  # Check if the source file exists
        shutil.copy(src_label_path, dst_label_path)

print("Images and labels have been copied to the corrected folders.")


Images and labels have been copied to the corrected folders.


In [6]:
#use only a subset of images to find thresholds

import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm  # Import tqdm for progress tracking
import random
import shutil

# Define the original folder paths
#original_folder_path_in = ''
#original_folder_path_out = ''

# Create new folder paths
#new_folder_path_in = ''
#new_folder_path_out = ''

# Create new directories
os.makedirs(new_folder_path_in, exist_ok=True)
os.makedirs(new_folder_path_out, exist_ok=True)

# Select a random subset of 5 images
input_tif_files = sorted([f for f in os.listdir(original_folder_path_in) if f.endswith('.tif')])
output_tif_files = sorted([f for f in os.listdir(original_folder_path_out) if f.endswith('.tif')])

random_subset_input = random.sample(input_tif_files, 5)
random_subset_output = random.sample(output_tif_files, 5)

# Copy the selected images to the new directories
for img in random_subset_input:
    shutil.copy(os.path.join(original_folder_path_in, img), new_folder_path_in)

for img in random_subset_output:
    shutil.copy(os.path.join(original_folder_path_out, img), new_folder_path_out)

# Set the new range for thresholds
red_thresh_list = list(range(62, 73, 1))  # 62 to 72 (inclusive)
green_thresh_list = list(range(34, 45, 1))  # 34 to 44 (inclusive)
blue_thresh = 20

# Initialize variables
errors = []
threshs = []
error = 0

# New input/output file lists
input_tif_files = sorted([f for f in os.listdir(new_folder_path_in) if f.endswith('.tif')])
output_tif_files = sorted([f for f in os.listdir(new_folder_path_out) if f.endswith('.tif')])

# Load true values from the CSV file
df = pd.read_csv('/work/users/d/p/dpguilba/FISH/2170_control_summary_names.csv')
true_greens = df['Actual MYC'][:5]  # Assuming you have enough entries
true_reds = df['Actual ERBB2'][:5]
true_red_greens = df['Actual MYC+ERBB2'][:5]

for redT in tqdm(red_thresh_list, desc='Red Thresholds'):
    for greenT in tqdm(green_thresh_list, desc='Green Thresholds', leave=False):
        red_thresh = redT
        green_thresh = greenT
        
        # Initialize current index for the images
        current_index = 0

        # Progress bar for the image processing loop
        with tqdm(total=len(input_tif_files), desc='Processing Images', leave=False) as pbar:
            for input_tif_file, output_tif_file in zip(input_tif_files, output_tif_files):
                input_tif_file_path = os.path.join(new_folder_path_in, input_tif_file)
                output_tif_file_path = os.path.join(new_folder_path_out, output_tif_file)
                
                input_tif_image = Image.open(input_tif_file_path)
                inarray = np.array(input_tif_image)
                
                output_tif_image = Image.open(output_tif_file_path)
                outarray = np.array(output_tif_image)
                
                outarray, mult_red, mult_green, mult_blue = mask(outarray, inarray)

                structure = np.ones((3, 3), dtype=int)
                labeled_out, num_comp = label(outarray, structure)

                comps, comp_red, comp_green, comp_blue = isolate_comps(num_comp, outarray, mult_red, mult_green, mult_blue, labeled_out)
            
                red_count, green_count, blue_count, red_green_count = count(num_comp, comp_red, comp_green, comp_blue, red_thresh, green_thresh, blue_thresh)
                
                true_red = true_reds[current_index]
                true_green = true_greens[current_index]
                true_red_green = true_red_greens[current_index]
                current_index += 1
                
                error += abs(true_red - red_count) + abs(true_green - green_count) + abs(true_red_green - red_green_count)

                pbar.update(1)  # Update the progress bar for each processed image

        # Store errors and thresholds
        errors.append(error)
        threshs.append([red_thresh, green_thresh])

min_value = min(errors)
#print(min_value)
min_index = errors.index(min_value)
thresh_min = threshs[min_index]
#print(thresh_min)

# Find the minimum value in the errors list
min_value = min(errors)

# Get the indices of the minimum value in the errors list
min_indices = [index for index, value in enumerate(errors) if value == min_value]

# Initialize max sum and corresponding thresh
max_sum = float('-inf')
best_thresh = None

# Iterate through the corresponding threshs for the minimum error indices
for i in min_indices:
    red_thresh, green_thresh = threshs[i]
    current_sum = red_thresh + green_thresh
    
    # Update if current sum is greater than max_sum
    if current_sum > max_sum:
        max_sum = current_sum
        best_thresh = (red_thresh, green_thresh)

# Output the results
print("Minimum error value:", min_value)
print("Maximum sum of thresholds:", max_sum)
print("Corresponding red_thresh and green_thresh:", best_thresh)

Red Thresholds:   0%|          | 0/11 [00:00<?, ?it/s]

Green Thresholds:   0%|          | 0/11 [00:00<?, ?it/s]

Processing Images:   0%|          | 0/5 [00:00<?, ?it/s]

Processing Images:   0%|          | 0/5 [00:00<?, ?it/s]

Processing Images:   0%|          | 0/5 [00:00<?, ?it/s]

...

Processing Images:   0%|          | 0/5 [00:00<?, ?it/s]

Processing Images:   0%|          | 0/5 [00:00<?, ?it/s]

Processing Images:   0%|          | 0/5 [00:00<?, ?it/s]

Processing Images:   0%|          | 0/5 [00:00<?, ?it/s]

Minimum error value: 1351.0
Maximum sum of thresholds: 96
Corresponding red_thresh and green_thresh: (62, 34)


In [7]:
#these thresholds represent a bare minimum to be counted as a true signal. Anything less can be considered noise

red_thresh, green_thresh = best_thresh
blue_thresh = 20

In [12]:
red_thresh=62, green_thresh = 34

34

In [16]:
#test one image, dont need to

#im = Image.open('') #(2048,2448) mask of 0s and 255s, output from MIA
outarray = np.array(im)

#im = Image.open('') #(2048,2448,3) rgb input image
inarray = np.array(im)

outarray, mult_red, mult_green, mult_blue = mask(outarray,inarray)

structure = np.ones((3, 3), dtype=int)
labeled_out, num_comp = label(outarray,structure)

comps,comp_red,comp_green,comp_blue = isolate_comps(num_comp,outarray,mult_red, mult_green, mult_blue,labeled_out)

red_count,green_count,blue_count,red_green_count = count(num_comp,comp_red,comp_green,comp_blue,red_thresh,green_thresh,blue_thresh)

print(f"Number of ecDNA with red: {red_count}")
print(f"Number of ecDNA with green: {green_count}")
print(f"Number of ecDNA with blue: {blue_count}")
print(f"Number of ecDNA with red and green: {red_green_count}")

Number of ecDNA with red: 1
Number of ecDNA with green: 3
Number of ecDNA with blue: 91
Number of ecDNA with red and green: 98


In [8]:
#generate probe predictions for control

#folder_path_out = ''
#folder_path_in = '' #not npz, just input images with all 3 channels

tif_files = [f for f in os.listdir(folder_path_in) if f.lower().endswith('.tif')]

input_tif_files = sorted([f for f in os.listdir(folder_path_in) if f.endswith('.tif')])
output_tif_files = sorted([f for f in os.listdir(folder_path_out) if f.endswith('.tif')])

columns = ['Image name','red_count','green_count','blue_count','red_green_count']
df = pd.DataFrame(columns=columns)
rows = []

for input_tif_file, output_tif_file in zip(input_tif_files, output_tif_files):
    
    input_tif_file_path = os.path.join(folder_path_in, input_tif_file)
    output_tif_file_path = os.path.join(folder_path_out, output_tif_file)
    
    tif_file_name = os.path.basename(input_tif_file)
    
    input_tif_image = Image.open(input_tif_file_path)
    inarray = np.array(input_tif_image) #(2048,2448,3)
    
    output_tif_image = Image.open(output_tif_file_path)
    outarray = np.array(output_tif_image) #(2048,2448) all 0s and 1s
    
    outarray, mult_red, mult_green, mult_blue = mask(outarray,inarray)

    structure = np.ones((3, 3), dtype=int)
    labeled_out, num_comp = label(outarray,structure)

    comps,comp_red,comp_green,comp_blue = isolate_comps(num_comp,outarray,mult_red, mult_green, mult_blue,labeled_out)

    red_count,green_count,blue_count,red_green_count = count(num_comp,comp_red,comp_green,comp_blue,red_thresh,green_thresh,blue_thresh)
    
    data = [tif_file_name,red_count,green_count,blue_count,red_green_count]
    row_df = pd.DataFrame([data], columns=columns)
    rows.append(row_df)
    df = pd.concat(rows, ignore_index=True)
    df.to_csv('FISH_data_2170_control_gt.csv', index=False)
    
        

In [9]:
#generate probe predictions for jq1

#folder_path_out = ''
#folder_path_in = '' #not npz, just input images with all 3 channels

tif_files = [f for f in os.listdir(folder_path_in) if f.lower().endswith('.tif')]

input_tif_files = sorted([f for f in os.listdir(folder_path_in) if f.endswith('.tif')])
output_tif_files = sorted([f for f in os.listdir(folder_path_out) if f.endswith('.tif')])

columns = ['Image name','red_count','green_count','blue_count','red_green_count']
df = pd.DataFrame(columns=columns)
rows = []

for input_tif_file, output_tif_file in zip(input_tif_files, output_tif_files):
    
    input_tif_file_path = os.path.join(folder_path_in, input_tif_file)
    output_tif_file_path = os.path.join(folder_path_out, output_tif_file)
    
    tif_file_name = os.path.basename(input_tif_file)
    
    input_tif_image = Image.open(input_tif_file_path)
    inarray = np.array(input_tif_image) #(2048,2448,3)
    
    output_tif_image = Image.open(output_tif_file_path)
    outarray = np.array(output_tif_image) #(2048,2448) all 0s and 1s
    
    outarray, mult_red, mult_green, mult_blue = mask(outarray,inarray)

    structure = np.ones((3, 3), dtype=int)
    labeled_out, num_comp = label(outarray,structure)

    comps,comp_red,comp_green,comp_blue = isolate_comps(num_comp,outarray,mult_red, mult_green, mult_blue,labeled_out)

    red_count,green_count,blue_count,red_green_count = count(num_comp,comp_red,comp_green,comp_blue,red_thresh,green_thresh,blue_thresh)
    
    data = [tif_file_name,red_count,green_count,blue_count,red_green_count]
    row_df = pd.DataFrame([data], columns=columns)
    rows.append(row_df)
    df = pd.concat(rows, ignore_index=True)
    df.to_csv('FISH_data_2170_jq1_gt.csv', index=False)

In [13]:
#generate summary files for control

import pandas as pd
import re
import os

# Load the CSV files
#jq1_df = pd.read_csv('')
#control_df = pd.read_csv('')
#ecDNA_df = pd.read_csv('')

# Step 1: Filter ecDNA_df to include only rows where 'Desc' contains 'JQ1_IC50_24h'  '2170_control'
#filtered_ecDNA_df = ecDNA_df[ecDNA_df['Desc'].str.contains('JQ1_IC50_24h', na=False)].copy()
filtered_ecDNA_df = ecDNA_df[ecDNA_df['Desc'].str.contains('2170_control', na=False)].copy()

#prefix = ''


# Step 2: Extract 'Site' from 'Image name' in jq1_df
def extract_site(image_name):
    base_name = image_name.split(prefix)[-1]
    name, ext = os.path.splitext(base_name)
    # match = re.search(r'NCIH2170_JC_JQ1_IC50_24h_(\d+)_Merge', image_name)
    # return str(match.group(1)) if match else None
    return name

#jq1_df['Site'] = jq1_df['Image name'].apply(extract_site)
control_df['Site'] = control_df['Image name'].apply(extract_site)



# Step 3: Extract 'Site' values from 'Desc' in filtered_ecDNA_df
#filtered_ecDNA_df['Site'] = filtered_ecDNA_df['Desc'].str.extract(r'NCIH2170_JC_JQ1_IC50_24h_(\d+)_Merge')[0]

# Convert 'Site' columns to strings
#jq1_df['Site'] = jq1_df['Site'].astype(str)
control_df['Site'] = control_df['Site'].astype(str)
filtered_ecDNA_df['Site'] = filtered_ecDNA_df['Site'].astype(str)

# Step 4: Merge jq1_df with filtered_ecDNA_df based on 'Site'
#merged_df = pd.merge(jq1_df, filtered_ecDNA_df, left_on='Site', right_on='Site')
merged_df = pd.merge(control_df, filtered_ecDNA_df, left_on='Site', right_on='Site')

# Print the column names in merged_df to check for discrepancies
print("Columns in merged_df:")
print(merged_df.columns.tolist())

# Check if the required columns exist in merged_df
required_columns = ['green_count', 'MYC+', 'red_count', 'ERBB2+', 'red_green_count', 'MYC+ERBB2+ ', 'blue_count','Unlabeled']
for col in required_columns:
    if col not in merged_df.columns:
        print(f"Column '{col}' not found in merged_df.")

# Continue with the rest of the code if all columns are present
if all(col in merged_df.columns for col in required_columns):
    # Step 5: Calculate the differences
    merged_df['Actual MYC'] = merged_df['MYC+']
    merged_df['Pred MYC'] = merged_df['green_count']
    merged_df['Actual ERBB2'] = merged_df['ERBB2+']
    merged_df['Pred ERBB2'] = merged_df['red_count']
    merged_df['Actual MYC+ERBB2'] = merged_df['MYC+ERBB2+ ']
    merged_df['Pred MYC+ERBB2'] = merged_df['red_green_count']
            
    
    # Step 6: Create the final DataFrame with desired columns
    #jq1_error_df = merged_df[['Actual MYC', 'Pred MYC', 'Actual ERBB2', 'Pred ERBB2', 'Actual MYC+ERBB2','Pred MYC+ERBB2']].copy()  # Ensure it's a copy
    control_error_df = merged_df[['Image name','Actual MYC', 'Pred MYC', 'Actual ERBB2', 'Pred ERBB2', 'Actual MYC+ERBB2','Pred MYC+ERBB2']].copy()  # Ensure it's a copy

    # Step 7: Save to a new CSV file
    #jq1_error_df.to_csv('2170_jq1_summary.csv', index=False)
    control_error_df.to_csv('2170_control_gt_summary_names.csv', index=False)

    print("The 'summary.csv' file has been created.")
else:
    print("Some required columns are missing. Please check the column names.")


Columns in merged_df:
['Image name', 'red_count', 'green_count', 'blue_count', 'red_green_count', 'Site', 'Desc', 'MYC+', 'ERBB2+', 'MYC+ERBB2+ ', 'Unlabeled', 'Total_ecDNA', 'HSR', 'multimers', 'Comments', 'Generated by']
The 'summary.csv' file has been created.


In [15]:
#generate summary files for jq1

import pandas as pd
import re
import os

# Load the CSV files
#jq1_df = pd.read_csv('')
#ecDNA_df = pd.read_csv('')

# Step 1: Filter ecDNA_df to include only rows where 'Desc' contains 'JQ1_IC50_24h'  '2170_control'
filtered_ecDNA_df = ecDNA_df[ecDNA_df['Desc'].str.contains('JQ1_IC50_24h', na=False)].copy()
#filtered_ecDNA_df = ecDNA_df[ecDNA_df['Desc'].str.contains('2170_control', na=False)].copy()

#prefix = 'NCIH2170_Control_'


# Step 2: Extract 'Site' from 'Image name' in jq1_df
def extract_site(image_name):
    # base_name = image_name.split(prefix)[-1]
    # name, ext = os.path.splitext(base_name)
    match = re.search(r'NCIH2170_JC_JQ1_IC50_24h_(\d+)_Merge', image_name)
    return str(match.group(1)) if match else None
    #return name

jq1_df['Site'] = jq1_df['Image name'].apply(extract_site)
#control_df['Site'] = control_df['Image name'].apply(extract_site)



# Step 3: Extract 'Site' values from 'Desc' in filtered_ecDNA_df
#filtered_ecDNA_df['Site'] = filtered_ecDNA_df['Desc'].str.extract(r'NCIH2170_JC_JQ1_IC50_24h_(\d+)_Merge')[0]

# Convert 'Site' columns to strings
jq1_df['Site'] = jq1_df['Site'].astype(str)
#control_df['Site'] = control_df['Site'].astype(str)
filtered_ecDNA_df['Site'] = filtered_ecDNA_df['Site'].astype(str)

# Step 4: Merge jq1_df with filtered_ecDNA_df based on 'Site'
merged_df = pd.merge(jq1_df, filtered_ecDNA_df, left_on='Site', right_on='Site')
#merged_df = pd.merge(control_df, filtered_ecDNA_df, left_on='Site', right_on='Site')

# Print the column names in merged_df to check for discrepancies
print("Columns in merged_df:")
print(merged_df.columns.tolist())

# Check if the required columns exist in merged_df
required_columns = ['green_count', 'MYC+', 'red_count', 'ERBB2+', 'red_green_count', 'MYC+ERBB2+ ', 'blue_count','Unlabeled']
for col in required_columns:
    if col not in merged_df.columns:
        print(f"Column '{col}' not found in merged_df.")

# Continue with the rest of the code if all columns are present
if all(col in merged_df.columns for col in required_columns):
    # Step 5: Calculate the differences
    merged_df['Actual MYC'] = merged_df['MYC+']
    merged_df['Pred MYC'] = merged_df['green_count']
    merged_df['Actual ERBB2'] = merged_df['ERBB2+']
    merged_df['Pred ERBB2'] = merged_df['red_count']
    merged_df['Actual MYC+ERBB2'] = merged_df['MYC+ERBB2+ ']
    merged_df['Pred MYC+ERBB2'] = merged_df['red_green_count']
            
    
    # Step 6: Create the final DataFrame with desired columns
    jq1_error_df = merged_df[['Image name','Actual MYC', 'Pred MYC', 'Actual ERBB2', 'Pred ERBB2', 'Actual MYC+ERBB2','Pred MYC+ERBB2']].copy()  # Ensure it's a copy
    #control_error_df = merged_df[['Image name','Actual MYC', 'Pred MYC', 'Actual ERBB2', 'Pred ERBB2', 'Actual MYC+ERBB2','Pred MYC+ERBB2']].copy()  # Ensure it's a copy

    # Step 7: Save to a new CSV file
    jq1_error_df.to_csv('2170_jq1_gt_summary_names.csv', index=False)
    #control_error_df.to_csv('2170_control_gt_summary_names.csv', index=False)

    print("The 'summary.csv' file has been created.")
else:
    print("Some required columns are missing. Please check the column names.")


Columns in merged_df:
['Image name', 'red_count', 'green_count', 'blue_count', 'red_green_count', 'Site', 'Desc', 'MYC+', 'ERBB2+', 'MYC+ERBB2+ ', 'Unlabeled', 'Total_ecDNA', 'HSR', 'multimers', 'Comments', 'Generated by']
The 'summary.csv' file has been created.
